In [1]:
from skimage import data, draw,color, data, restoration
from skimage import transform, util
from scipy import misc, fftpack, signal
import numpy as np
from skimage import filters, color
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
import matplotlib.animation as animation
from scipy.misc import imresize
from scipy import ndimage as nd
import scipy
#%matplotlib inline

In [2]:
rgb =  plt.imread('agbar.png')
img = color.rgb2gray(rgb)
img = util.img_as_float(img)
fig = plt.figure(1) 
plt.imshow(rgb)
plt.title('Original Image')
plt.show()

In [3]:
def gradient(img):
    img = color.rgb2gray(img)
    gx, gy = np.gradient(img)
    grad = np.sqrt(gx*gx + gy*gy)
    return grad
grad = gradient(img)

fig = plt.figure(2) 
plt.title('Contorns imatge')
plt.imshow(grad, cmap="gray" )
plt.show()

In [4]:
#normalitzacio d'imatge
def normalitzar(imatge):
    return np.real((imatge - np.min(imatge)) * 256 / (np.max(imatge) - np.min(imatge)))

In [5]:

#Calcul matriu d'energia que te cada zona
def energia(imatge):
    height,width = imatge.shape
    energia=np.zeros(imatge.shape)
    #asignem la primera linea igual que la imatge que tenim
    energia[0]=imatge[0]   
    for x in range(1,height):
        for y in range(0,width):
            if y == 0:
                min_val = min( energia[x-1, y], energia[x-1, y+1] )
            elif y < width - 2:
                min_val = min( energia[x-1, y], energia[x-1, y+1] )
                min_val = min( min_val, energia[x-1, y-1] )
            else:
                min_val = min( energia[x-1, y], energia[x-1, y-1] )
            energia[x,y] = imatge[x,y] + min_val
    return energia
            
    
    

In [6]:
grad=normalitzar(grad)
E = energia(grad)
fig = plt.figure(3) 
plt.title('Acumulacio energia')
plt.imshow(E, cmap="gray" )
plt.show()



In [6]:
#SELECCIÓ INTERACTIVA DE LA ZONA A BORRAR
class get_mouse_click():
    """Mouse interaction interface for radial distortion removal.

    """
    def __init__(self, img):
      height, width = img.shape[:2]
      self.figure = plt.imshow(img, extent=(0, width, height, 0))
      plt.gray()
      plt.title('select the object to remove')
      plt.xlabel('Select sets of  points with left mouse button,\n'
                 'click right button to close the polygon.')
      plt.connect('button_press_event', self.button_press)
      plt.connect('motion_notify_event', self.mouse_move)

      self.img = np.atleast_3d(img)
      self.points = []
      self.centre = np.array([(width - 1)/2., (height - 1)/2.])

      self.height = height
      self.width = width

      self.make_cursorline()
      self.figure.axes.set_autoscale_on(False)

      plt.show()
      plt.close()

    def make_cursorline(self):
        self.cursorline, = plt.plot([0],[0],'r:+',
                                    linewidth=2,markersize=15,markeredgecolor='b')

    def button_press(self,event):
        """Register mouse clicks.

        """
        if (event.button == 1 and event.xdata and event.ydata):
            self.points.append((event.xdata,event.ydata))
            print "Coordinate entered: (%f,%f)" % (event.xdata, event.ydata)

            #if len(self.points) % 2 == 0:
            plt.gca().lines.append(self.cursorline)
            self.make_cursorline()

        if (event.button != 1):
            #print "pepito: " ,self.points
            self.points.append((self.points[0][0],self.points[0][1]))
            plt.close()
            return self.points
            #qui
            #print "Removing distortion..."
            #plt.gca().lines = []
            #plt.draw()
            #self.remove_distortion()
            #self.points = []

    def mouse_move(self,event):
        """Handle cursor drawing.

        """
        #pt_sets, pts_last_set = divmod(len(self.points),5)
        #print pts_last_set
        pts_last_set=len(self.points)
        #print pts_last_set2
        pts = np.zeros((pts_last_set+1,2))
        if pts_last_set > 0:
            # Line follows up to 3 clicked points:
            pts[:pts_last_set] = self.points[-pts_last_set:]
            # The last point of the line follows the mouse cursor
        pts[pts_last_set:] = [event.xdata,event.ydata]
        #print pts
        self.cursorline.set_data(pts[:,0], pts[:,1])
        plt.draw()


def compute_mask(width,height,polygon):
    
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    mask = np.array(img)
    mask100 = 100*np.ones([mask.shape[0],mask.shape[1]])
    mask=mask*101
    mask = 1. - mask; # switch 0s and 1s
    return mask

In [7]:
# El calcul del tall vertical hem de buscar el cami amb cost minim. BACKTRACKING
# Aquest tall pot ser vertical o horitzontal, es un camí de pixels connectats. En el nostre cas l'apliquem vertical
# La función de importancia / energía valora un píxel mediante la medición de su contraste con sus píxeles vecinos.
# Des de la imatge en escala de grisos retornarem el tall vertical mes baix com a llista de pixels

def tall_vertical(grad,cost):
    # Anem fins al -1 perque aixi es quedara en el 0, que tambe el necessitem.
    width, height = grad.shape
    
    min_val = 1e1000
    path = []
    for y in range(height):
        if cost[width-1,y] < min_val:
            min_val = cost[width-1,y]
            min_ptr = y
            
    pos = (width-1, min_ptr)
    path.append(pos) 
    while pos[0] != 0:
        val = cost[pos] - grad[pos]
        x,y = pos
        if y == 0:
            if val == cost[x-1,y+1]:
                pos = (x-1,y+1)
            else:
                pos = (x-1,y)
        elif y <= height - 2:
            if val == cost[x-1,y+1]:
                pos = (x-1,y+1)
            elif val == cost[x-1,y]:
                pos = (x-1,y)
            else:
                pos = (x-1,y-1)
        else:
            if val == cost[x-1,y]:
                pos = (x-1,y)
            else:
                pos = (x-1,y-1)
                
        path.append(pos)  
    return path 


In [8]:
def borrar_cami (img, path):
    #modifiquem la funció perque pugui borrar tant de la imatge com la mascara
    height, width = img.shape[:2]
    if(len(img.shape) > 2):
        img_retallada = np.zeros((height, width-1, img.shape[2]))
    else:
        img_retallada = np.zeros((height, width-1))
    path_cpy = set(path)
    visitat = set()
    #print path[1]
    for x in range(height):
        for y in range(width):
            if x not in visitat and (x,y) not in path_cpy:
                img_retallada[x,y] = img[x,y]
            elif (x,y) in path_cpy:
                visitat.add(x)
            else:
                img_retallada[x,y-1] = img[x,y]
    #print img_retallada.shape
    return img_retallada;

def mark_seam (mat, path, mark_as='red'):
    #Dono maxim valor al canal corresponent al color escollit a tots els pixels del path que ens marca el cami a borrar
    for i in path:
        if mark_as == 'red':
            mat[i] = (255,0,0)
        elif mark_as == 'green':
            mat[i] = (0,255,0)
        elif mark_as == 'blue':
            mat[i] = (0,0,255)
        elif mark_as == 'white':
            mat[i] = (255,255,255)
        elif mark_as == 'black':
            mat[i] = (0,0,0)
    
    return mat;                 


In [9]:
def apply_mask(img, mask):    
    masked = img.copy()

    for x in xrange(mask.shape[0]):
        for y in xrange(mask.shape[1]):
            if mask[x,y] == -100:
                masked[x,y] = mask[x,y]
    return masked

In [10]:
rgb =  plt.imread('agbar.png')
img = color.rgb2gray(rgb)
img = util.img_as_float(img)
width, height = img.shape
rdi = get_mouse_click(rgb)
polygon = rdi.points
mascara = compute_mask(height, width, polygon)
#dono maxim valor a la zona on hi ha l'area seleccionada
grad = apply_mask(gradient(img), mascara)
E = energia(grad)
fig = plt.figure(2) 
plt.title('Contorns imatge')
plt.imshow(grad, cmap="gray" )
plt.show()
E = energia(grad)
fig = plt.figure(3) 
plt.title('Acumulacio energia')
plt.imshow(E, cmap="gray" )
plt.show()

Coordinate entered: (258.333333,154.166667)
Coordinate entered: (220.833333,233.333333)
Coordinate entered: (214.583333,437.500000)
Coordinate entered: (300.000000,441.666667)
Coordinate entered: (304.166667,293.750000)
Coordinate entered: (283.333333,162.500000)
Coordinate entered: (256.250000,154.166667)


In [15]:
def carving(imatge,mascara):
    path = []
    #path= tall_vertical(h, llistaBacktracking[2], E, path)    
    #borrar_cami(grad, path)
    #rgb =  plt.imread('countryside.jpg')
    
    rgb = imatge
    seams = []

    for t in range(100):
        print "iteration "+str(t),"\tseam carved...", rgb.shape
        path = []
        grad = apply_mask(gradient(rgb), mascara)
        E = energia(grad)
        path = tall_vertical(grad,E)
        rgb = mark_seam(rgb, path, 'red')
        seams.append([plt.imshow(rgb.astype('uint8'))])
        rgb = borrar_cami(rgb, path)
        mascara = borrar_cami(mascara, path)
        
    return rgb,seams;

In [19]:
rgb =  plt.imread('agbar.png')
img = color.rgb2gray(rgb)
img = util.img_as_float(img)
width, height = img.shape
# Selecciono l'area a borrar
selected_area = get_mouse_click(rgb)
polygon = selected_area.points

# Calculo la mascara
mascara = compute_mask(height, width, polygon)
#print mask.shape








Coordinate entered: (258.333333,150.000000)
Coordinate entered: (229.166667,220.833333)
Coordinate entered: (218.750000,331.250000)
Coordinate entered: (320.833333,331.250000)
Coordinate entered: (262.500000,147.916667)


In [20]:


imatge_reduida, seams = carving(rgb, mascara)
plt.title('Imatge Retallada')
plt.imshow(imatge_reduida)
fig = plt.figure(1)



iteration 0 	seam carved... (800, 600, 3)
iteration 1 	seam carved... (800, 599, 3)
iteration 2 	seam carved... (800, 598, 3)
iteration 3 	seam carved... (800, 597, 3)
iteration 4 	seam carved... (800, 596, 3)
iteration 5 	seam carved... (800, 595, 3)
iteration 6 	seam carved... (800, 594, 3)
iteration 7 	seam carved... (800, 593, 3)
iteration 8 	seam carved... (800, 592, 3)
iteration 9 	seam carved... (800, 591, 3)
iteration 10 	seam carved... (800, 590, 3)
iteration 11 	seam carved... (800, 589, 3)
iteration 12 	seam carved... (800, 588, 3)
iteration 13 	seam carved... (800, 587, 3)
iteration 14 	seam carved... (800, 586, 3)
iteration 15 	seam carved... (800, 585, 3)
iteration 16 	seam carved... (800, 584, 3)
iteration 17 	seam carved... (800, 583, 3)
iteration 18 	seam carved... (800, 582, 3)
iteration 19 	seam carved... (800, 581, 3)
iteration 20 	seam carved... (800, 580, 3)
iteration 21 	seam carved... (800, 579, 3)
iteration 22 	seam carved... (800, 578, 3)
iteration 23 	seam ca

In [18]:
#run animation
ani = animation.ArtistAnimation(fig, seams, interval=100, blit=True, repeat_delay=1000);
plt.show();
plt.title('Acumulacio energia')
plt.imshow(imatge_reduida)
plt.show()



In [ ]:
######################Comparacio RESULTATS
plt.subplot(1,2,1)
plt.imshow(plt.imread('agbar.png'))
plt.title('Original')
plt.subplot(1,2,2)
plt.imshow(rgb)
plt.title('Reduccion 100 iteracions')
plt.gcf().set_size_inches((15,15))